In [ ]:
import os
import cv2
import pandas as pd

import numpy as np
from skimage.feature import graycomatrix, graycoprops

In [1]:
# Define the directories
# Store your directories in a "data" folder in your working directory
CURRENT_DIR = os.getcwd()
TRAIN_DIR = CURRENT_DIR + "//data//train"
TEST_DIR = CURRENT_DIR + "//data//test"

# Create empty DataFrames for the train and test data
train_df = pd.DataFrame()
test_df = pd.DataFrame()

# Define a function to read and resize an image
def read_and_resize_image(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (128, 128))
    # Convert the resized image to grayscale (2D array)
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    return gray_image  # Now, the image is a 2D NumPy array (grayscale)

def process_image(DIR):

    #Initiate Dataframe
    df = pd.DataFrame()

    # Create an empty list to store the new rows
    new_rows = []

    # Iterate over the training directory and read in the images
    for subdir in os.listdir(DIR):
        subdir_path = os.path.join(DIR, subdir)
        for image in os.listdir(subdir_path):
            image_path = os.path.join(subdir_path, image)

            # Read and resize the image
            resized_image = read_and_resize_image(image_path)

            # Create a new row for the train DataFrame
            new_row = {
                "image": resized_image,
                "label": {
                    "NonDemented": 0,
                    "VeryMildDemented": 1,
                    "MildDemented": 2,
                    "ModerateDemented": 3
                }[subdir]
            }

            # Add the new row to the list
            new_rows.append(new_row)

    # Concatenate the new rows to the train DataFrame
    df = pd.concat([train_df, pd.DataFrame(new_rows)])

    return df

# Process train and test DataFrames
train_df = process_image(TRAIN_DIR)
test_df = process_image(TEST_DIR)

In [15]:
# Get the image data from the DataFrames
train_images = train_df['image'].values
test_images = test_df['image'].values

# Define the function to compute GLCM features for multiple distances and angles
def compute_glcm_features(images, distances, angles, properties):
    glcm_features = []
    for image in images:
        image_glcm_features = []
        for distance in distances:
            for angle in angles:
                # Calculate GLCM for each image, distance, and angle
                glcm = graycomatrix(image, [distance], [angle], levels=256, symmetric=True, normed=True)
                # Compute the specified GLCM properties
                texture_features = [graycoprops(glcm, prop)[0, 0] for prop in properties]
                image_glcm_features.extend(texture_features)
        # Append the computed features for all distances and angles to the result list
        glcm_features.append(image_glcm_features)
    return glcm_features

# Define the GLCM properties you want to compute
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# Define the distances and angles
distances = [1, 2, 3]
angles = [0, np.pi/4, np.pi/2]

# Compute GLCM features for train and test images
train_glcm_features = compute_glcm_features(train_images, distances, angles, properties)
test_glcm_features = compute_glcm_features(test_images, distances, angles, properties)

# Create DataFrames for train and test features
X_train = pd.DataFrame(train_glcm_features, columns=[f"{prop}_d{distance}_a{angle}" for prop in properties for distance in distances for angle in angles])
X_test = pd.DataFrame(test_glcm_features, columns=[f"{prop}_d{distance}_a{angle}" for prop in properties for distance in distances for angle in angles])


In [16]:
# Extract labels from train and test DataFrames
y_train = train_df["label"]
y_test = test_df["label"]

In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5120, 54)
(5120,)
(5160, 54)
(5160,)


In [25]:
directory_path = "processed_data/glcm/"

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Save the train and test DataFrames
X_train.to_pickle("processed_data/glcm/X_train.pkl")
y_train.to_pickle("processed_data/glcm/y_train.pkl")
X_test.to_pickle("processed_data/glcm/X_test.pkl")
y_test.to_pickle("processed_data/glcm/y_test.pkl")